# Sampling from SVG VAE

This is a quick notebook I ([Allison Parrish](http://www.decontextualize.com/) made in order to sample glyphs from [SVG VAE](https://magenta.tensorflow.org/svg-vae), a generative model for scalable vector graphics programmed and trained by [Rapha Gontijo Lopes](https://raphagl.com/). ([Read the paper](https://arxiv.org/abs/1904.02632).)

The code is largely based on [the example notebook](https://github.com/magenta/magenta-demos/blob/master/colab-notebooks/vae_svg_decoding.ipynb) included in the Magenta repository, with a few tweaks I made in order to make the code and content compatible with the tutorials in a class I'm teaching at [NYU ITP]() called [Material of Language](https://github.com/aparrish/material-of-language/).

In this notebook, I'm only demonstrating the model's ability to generate "fonts" (i.e., sets of glyphs), and to generate interpolations between different fonts. My goal was to make it possible for someone without a lot of knowledge about programming or machine learning to step through the notebook, play around with the code a bit, and produce something interesting.

This is a Google Colab notebook. I don't usually like using Google Colab, but I also don't want to have to help all of my students install all of the finicky dependencies on their own computers. So here we are.

> Note for my students: Colab notebooks don't run on your own computer—they run on a computer that Google provides to you for free for a few hours. If the notebook is inactive for a while, the system deletes the machine—and your work. The notebook file itself is saved to Google Drive, but any files that you save will be deleted when the machine is recycled. Also note that you'll need to *redownload data and re-install packages* whenever you open the notebook (if the machine has timed out since the last time you used it). 

For a quick explanation of how the model works and its architecture, I recommend reading [the clear and concise blog post](https://magenta.tensorflow.org/svg-vae) that the authors wrote about the model for Magenta.

## Download the model and associated data

Before we get started programming, we need to download the pre-trained model. The authors made this available online; run the cell below to download it:

In [ ]:
!curl -L -O https://storage.googleapis.com/magentadata/models/svg_vae/svg_vae.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  776M  100  776M    0     0  90.6M      0  0:00:08  0:00:08 --:--:-- 95.1M


The models are stored in a gzipped tar file—uncompress it by running the cell below:

In [ ]:
!tar xvfz svg_vae.tar.gz

usr/local/google/home/iraphael/dev/svg_vae/
usr/local/google/home/iraphael/dev/svg_vae/svg_decoder_internal/
usr/local/google/home/iraphael/dev/svg_vae/svg_decoder_internal/model.ckpt-300000.data-00000-of-00002
usr/local/google/home/iraphael/dev/svg_vae/svg_decoder_internal/checkpoint
usr/local/google/home/iraphael/dev/svg_vae/svg_decoder_internal/model.ckpt-300000.data-00001-of-00002
usr/local/google/home/iraphael/dev/svg_vae/svg_decoder_internal/model.ckpt-300000.meta
usr/local/google/home/iraphael/dev/svg_vae/svg_decoder_internal/model.ckpt-300000.index
usr/local/google/home/iraphael/dev/svg_vae/image_vae_internal/
usr/local/google/home/iraphael/dev/svg_vae/image_vae_internal/model.ckpt-100000.meta
usr/local/google/home/iraphael/dev/svg_vae/image_vae_internal/model.ckpt-100000.data-00000-of-00002
usr/local/google/home/iraphael/dev/svg_vae/image_vae_internal/model.ckpt-100000.data-00001-of-00002
usr/local/google/home/iraphael/dev/svg_vae/image_vae_internal/checkpoint
usr/local/google

You'll also need the following two files, which are used to "un-normalize" the SVG paths that come out of the model. (Thanks to Rapha for giving me access to these files!)

In [ ]:
!curl -L -O https://github.com/aparrish/material-of-language/raw/master/svg_vae_data/mean.npz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   158  100   158    0     0    793      0 --:--:-- --:--:-- --:--:--   793
100   160  100   160    0     0    375      0 --:--:-- --:--:-- --:--:--  7619


In [ ]:
!curl -L -O https://github.com/aparrish/material-of-language/raw/master/svg_vae_data/stdev.npz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   159  100   159    0     0    836      0 --:--:-- --:--:-- --:--:--   836
100   160  100   160    0     0    418      0 --:--:-- --:--:-- --:--:--   418


## Dependencies

We need the `tensor2tensor` library and the `magenta` library. Hopefully both of these will install without trouble in Colab:

In [ ]:
!pip install tensor2tensor==1.14.1

     |████████████████████████████████| 1.6MB 4.9MB/s 
     |████████████████████████████████| 307kB 30.8MB/s 
     |████████████████████████████████| 5.3MB 30.0MB/s 
     |████████████████████████████████| 655kB 46.7MB/s 
     |████████████████████████████████| 184kB 43.7MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 368kB 36.5MB/s 
     |████████████████████████████████| 983kB 45.2MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 235kB 43.9MB/s 
  Created wheel for pypng: filename=pypng-0.0.20-cp36-none-any.whl size=67161 sha256=4e2ae25c1772a61a9a5879eafdb396d9c5f4c8cc2c00bcb24a218482684c1538
  Stored in directory: /root/.cache/pip/wheels/41/6b/ef/0493b536b6d4722c2ae9486691b1d49b922b9877922beeabb3
  Created wheel for bz2file: filename=bz2file-0.98-cp36-none-any.whl size=6884 sha256=01daa803c687228f12ffdbaf70ac9cd3f108763cb1d46f47b12e9a47d28b6fb5
  Stored in directory: /root/.cache/p

In [ ]:
!pip install magenta==1.3.2

     |████████████████████████████████| 1.6MB 4.1MB/s 
     |████████████████████████████████| 2.3MB 31.4MB/s 
     |████████████████████████████████| 8.3MB 35.5MB/s 
     |████████████████████████████████| 110.5MB 50kB/s 
     |████████████████████████████████| 5.6MB 10.1MB/s 
     |████████████████████████████████| 204kB 44.5MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 1.6MB 40.0MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 655kB 36.6MB/s 
     |████████████████████████████████| 1.4MB 33.5MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 829kB 32.4MB/s 
     |████████████████████████████████| 63.8MB 67kB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 153kB 40.4MB/s 
     |████████████████████████████████| 215kB 44.1MB/s 
    

We also need Flat and Bezmerizing so we can draw the SVG commands that come out of the model. (There are other ways to do this, of course, but I like having the flexibility of Flat and Bezmerizing.)

In [ ]:
!pip install flat

     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for flat: filename=Flat-0.3.1-cp36-none-any.whl size=54586 sha256=c1a90745423a9ab561a99fa8c54e2a67da39fabff3192809fa26b7506bdaf528
  Stored in directory: /root/.cache/pip/wheels/55/2e/c5/1842eeb06871d1e2cbf96ef75b88daccd6e487ca3f50d7ba3b
Successfully built flat


In [ ]:
!pip install https://github.com/aparrish/bezmerizing/archive/master.zip

     - 256kB 849kB/s
  Created wheel for bezmerizing: filename=bezmerizing-0.1.0-cp36-none-any.whl size=15816 sha256=e2036a4fd2d546644c9b212cee31e5eeceac31f5fb69ece55fca90cbec5e4a4a
  Stored in directory: /tmp/pip-ephem-wheel-cache-5ga5bt8m/wheels/93/43/9b/8acebe44e4ac5fc93427a03a09bd7cd8110979c5701c77c7f8
Successfully built bezmerizing


## Code preliminaries

The following cell makes available various libraries that the rest of the code needs to function, including the Flat and Bezmerizing libraries that we'll use to draw the paths generated by the model.

In [ ]:
from __future__ import print_function

import copy
import logging
import numpy as np
import re
import tensorflow as tf
import warnings

from tensor2tensor.utils import registry
from magenta.models import svg_vae
from magenta.models.svg_vae import svg_utils
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib

# for drawing the paths
from flat import document, shape, rgb, svg
from bezmerizing import Path

tfe = tf.contrib.eager
Modes = tf.estimator.ModeKeys

tf.enable_eager_execution()
tf.get_logger().setLevel('WARN')

logging.getLogger("mlperf_compliance").setLevel(logging.ERROR)
warnings.simplefilter("ignore")

from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.







This function loads and returns the pre-trained model from disk:

In [ ]:
def initialize_model_svg(problem_name, hparam_set, hparams, model_name,
                     ckpt_dir, split=Modes.TRAIN):
  """Returns an initialized model, dataset iterator and hparams."""
  tf.reset_default_graph()
  
  # create hparams and get glyphazzn problem definition
  hparams = trainer_lib.create_hparams(hparam_set, hparams,
                                       problem_name=problem_name)
  problem = registry.problem(problem_name)
  
  # get model definition
  ModelClass = registry.model(model_name)
  model = ModelClass(hparams, mode=Modes.PREDICT,
                     problem_hparams=hparams.problem_hparams)
  
  # the model apparently needs to be "fed" a sample before you can
  # initialize it from the checkpoint. as far as I can tell, it
  # doesn't matter what's in this sample—it can be completely made up.
  # I'm not sure if I got these dimensions right, not sure if it matters?
  features = {}
  features['bottleneck'] = tf.random_normal([64, 32])
  features['targets_cls'] = [[11]*64]
  features['inputs'] = tf.zeros([64, 51, 1, hparams.problem.feature_dim])
  features['targets_psr'] = tf.zeros([64, 1, 64*64, 1])
  features['targets'] = tf.zeros([64, 51, 1, hparams.problem.feature_dim])
  output, extra_losses = model(features)
  model.initialize_from_ckpt(ckpt_dir)  # initializes ops
  
  return model, hparams

... and this function loads and returns the numpy arrays that we need in order to un-normalize the paths:

In [ ]:
def get_means_stdevs():
  with tf.gfile.Open('mean.npz', 'rb') as f:
    mean_npz = np.load(f)
  with tf.gfile.Open('stdev.npz', 'rb') as f:
    stdev_npz = np.load(f)
  return mean_npz, stdev_npz

## Generating glyphs from the model

Okay, with all that out of the way, we can actually get around to generating stuff with the model. First, load the model like so:

In [ ]:
svg_model, svg_hparams = initialize_model_svg(
    problem_name='glyph_azzn_problem',
    hparam_set='svg_decoder',
    hparams=(
        "vae_ckpt_dir=usr/local/google/home/iraphael/dev/svg_vae/image_vae_external/,"
        "vae_hparam_set=image_vae,mix_temperature=0.0001,gauss_temperature=0.0001"
    ),
    model_name='svg_decoder',
    ckpt_dir='usr/local/google/home/iraphael/dev/svg_vae/svg_decoder_external/')






Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use `tf.random.categorical` instead.
Instructions for updating:
Use 

We want to use the model to generate SVG paths for letterforms. To do this, you call the model's `.infer()` method, with a vector as a parameter. The model "decompresses" this vector into a sequence of SVG commands. Because this is a generative model, you can pass in arbitrary numbers for this vector—even random numbers (using a normal distribution). SVG paths generated from similar vectors will have similar characteristics.

The function in the cell below calls the model's `.infer()` method with the specified vector. (This vector should have 32 dimensions, which is the size of the model's latent variable. It'll work best with a number that is sampled from a normal distribution.) It generates one vector for every letter in the model's vocabulary (0-9, A-Z, a-z).

In [ ]:
def infer_alphabet(z):
    batch_size = 62 # total number of classes
    # dimensions of pretrained model's latent space (I just happen to know this)
    zdim = 32
    # using the same z value for each letter in the alphabet
    bottleneck = tf.tile([z.astype(np.float32)], [batch_size, 1])
    # the .infer() method requires an "example" sample from the dataset.
    # here I just create a dummy record. the only important values are
    # `bottleneck` (the value for the latent space) and `targets_cls` (the
    # classes that we want to generate)
    features = {}
    features['bottleneck'] = bottleneck
    features['targets_cls'] = [i for i in range(batch_size)]
    features['inputs'] = tf.zeros([batch_size, 51, 1,
                                   svg_hparams.problem.feature_dim])
    features['targets'] = tf.zeros([batch_size, 51, 1,
                                    svg_hparams.problem.feature_dim])
    features['targets_psr'] = tf.zeros([batch_size, 1, 64*64, 1])
    out = svg_model.infer(features, decode_length=0)
    return out['outputs']

Run the function like so. (The `np.random.randn()` function generates a random vector with the specified number of dimensions following a normal distribution.)

In [ ]:
alphabet = infer_alphabet(np.random.randn(32))

The resulting value is a big old weird tensor that isn't very valuable to us on its own. ("Tensor" btw is essentially a fancy word for "a list with other lists in it, and those lists might have other lists in them to, and so forth.") But each row contains an encoded version of an SVG command. There's one row for each letter in the model's vocabulary.

In [ ]:
alphabet

<tf.Tensor: id=38537, shape=(62, 51, 1, 10), dtype=float32, numpy=
array([[[[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
          -9.37601388e-01,  3.10494566e+00,  1.49910021e+00]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -2.16766559e-02,  3.38506624e-02,  1.61985800e-01]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           7.33388364e-02,  3.83214235e-01,  1.48813695e-01]],

        ...,

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -7.41805509e-02, -1.42593428e-01, -1.54692113e-01]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -1.53501377e-01, -6.26851022e-02, -1.61252558e-01]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -9.77352411e-02, -1.32421162e-02, -1.21311516e-01]]],


       [[[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
          -5.05234420e-01,  3.05760145e+00,  1.52515888e+00]],

       

To display the character generated by the model, we need to convert that encoded value to an actual SVG command. Even more useful to us would be a Bezmerizing `Path` object, which we can use to draw the glyph to the screen using Flat.

The function below takes the tensor returned from `infer_alphabet()` and returns a list of `Path` objects for each row in the tensor. (This function makes use of various functions that are distributed with the source code of the model in its `svg_utils` module.)

In [ ]:
def tensor_to_paths(out):
    mean_npz, stdev_npz = get_means_stdevs()
    all_paths = []
    for i in range(out.shape[0]):
        tensor = tf.expand_dims(out[i], 0)
        # "unnormalize" the tensor (using values calculated from the
        # pre-processed data set)
        tensor = (tensor * stdev_npz) + mean_npz
        vector = tf.squeeze(svg_utils.make_simple_cmds_long(tensor), [0, 2])
        path_parts = []
        # convert each row in this vector to an SVG command
        for vec in vector:
            cmd = svg_utils.vector_to_cmd(vec, categorical=True,
                                          return_floats=True)
            # if this is the end of the string, break out of the loop
            is_eos = np.argmax(vec[:20]) == 0
            if is_eos:
                break
            if len(cmd) > 0:
                cmd_formatted = cmd[0] + ' ' + ' '.join(
                    [f'{c:.5f}' for c in cmd[1:]])
                path_parts.append(cmd_formatted)
        path_str = ' '.join(path_parts)
        path = Path(svg.parsepath(path_str.encode('utf8')))
        all_paths.append(path)
    return all_paths

Finally, the `generate_alphabet()` function combines these two, returning the paths for an alphabet generated from a single vector:

In [ ]:
def generate_alphabet(z):
    return tensor_to_paths(infer_alphabet(z))

So now we can easily generate an entire alphabet of paths:

In [ ]:
alphabet_paths = generate_alphabet(np.random.randn(32))

Take a look at one of the paths:

In [ ]:
alphabet_paths[10]

Path([moveto(x=14.3238, y=10.5709), lineto(x=15.6100, y=10.5924), lineto(x=16.3921, y=23.0784), lineto(x=18.0033, y=23.0733), lineto(x=17.9627, y=23.5642), lineto(x=13.0283, y=23.5530), lineto(x=13.1349, y=23.0757), lineto(x=14.7853, y=23.0535), lineto(x=14.2406, y=19.1435), lineto(x=8.5528, y=19.1443), lineto(x=5.8543, y=22.5063), lineto(x=7.4960, y=23.3516), lineto(x=7.3342, y=23.7764), lineto(x=2.8617, y=23.7359), lineto(x=2.9566, y=23.3116), lineto(x=4.7223, y=23.0344), curveto(x1=6.1885, y1=20.0370, x2=9.8902, y2=16.9675, x=10.1654, y=11.4301), lineto(x=12.3208, y=11.0556), moveto(x=11.4555, y=12.4222), lineto(x=6.9433, y=19.5442), lineto(x=13.3487, y=19.5442), lineto(x=11.6169, y=12.8175)])

The `generate_alphabet()` function returns a list of paths. In this list, the index for a particular character corresponds to that character's position in the sequence of values 0-9, A-Z, a-z. The function below takes a character and returns its position in this sequence (if applicable):


In [ ]:
def c2i(ch):
    lookup = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
    return lookup.index(ch)

For example, the index of the capital letter `P`:

In [ ]:
c2i('P')

25

The following cell puts all of this together, and draws the glyph in this generated alphabet for the letter `P`.

In [ ]:
glyph = alphabet_paths[c2i('P')]
page = document(80, 80, 'mm').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(128, 128, 128))
page.place(pen.path(glyph.scale(4)))
show(page)

The following example draws all 62 glyphs in the alphabet in a grid, so you can preview the results. (Each time you run the cell, it will generate a new alphabet.)

In [ ]:
alphabet_paths = generate_alphabet(np.random.randn(32))
width = 100
height = 100
page = document(width, height, 'mm').addpage()
pen = shape().nostroke().fill(rgb(40, 40, 40))
grid_size = 8
cell_size = width / grid_size
for i in range(grid_size):
    for j in range(grid_size):
        x = i * cell_size
        y = j * cell_size
        idx = (j * grid_size) + (i % grid_size)
        if idx < len(alphabet_paths):
            this_glyph = alphabet_paths[idx]
            page.place(pen.path(this_glyph.scale(0.6).translate(x, y)))
show(page)

## Typesetting with SVG-VAE glyphs

Now we can draw single glyphs to the screen, which is fantastic! If we want to actually typeset some text, it's a bit more tricky. The model doesn't tell us each character's advance, so we'll have to guess! The characters are also generated in the middle of their canvas, so we need to correct for the fact that some narrow characters (like, say, `i`) will need to have their path data shifted over in order to get rid of empty space on the left hand side of the character.

For simplicity's sake in this section, I'm going to focus on setting *single lines* of text.

First, an example string:

In [ ]:
s = "456 Quartzy Foxes"

And then we'll generate an alphabet:

In [ ]:
alpha = generate_alphabet(np.random.randn(32))

The function below calculates the minimum and maximum x-coordinates of every command in the path. This gives us both the amount we need to shift the glyph in order to eliminate empty space to the left of the glyph, and how far we need to advance the cursor after drawing the glyph:

In [ ]:
def x_extent(path):
    all_x = [getattr(cmd, 'x', 0) for cmd in path.commands]
    return min(all_x), max(all_x)

If this works, we should be able to draw a rectangle around a character. Let's try it out:

In [ ]:
ch = 'j'
min_x, max_x = x_extent(alpha[c2i(ch)])
page = document(32, 32, 'mm').addpage()
rect_pen = shape().stroke(rgb(0, 255, 0)).nofill()
letter_pen = shape().nostroke().fill(rgb(40, 40, 40))
page.place(rect_pen.rectangle(min_x, 0, max_x - min_x, 32))
page.place(letter_pen.path(alpha[c2i(ch)]))
show(page)

Working from the example code in the [manipulating font data notebook](https://github.com/aparrish/material-of-language/blob/master/manipulating-font-data.ipynb), we now have everything we need to typeset a single line of text. I've commented the source code below:

In [ ]:
page = document(300, 50, 'mm').addpage()
pen = shape().nostroke().fill(rgb(40, 40, 40))
cx = 0 # current "cursor" position
kern = 1 # to modify spacing between characters
for ch in s:
    # if it's a space, just advance
    if ch == ' ':
        advance = 5 # maybe?
    # otherwise...
    else:
        glyph_path = alpha[c2i(ch)] # get path for this character
        min_x, max_x = x_extent(glyph_path) # find min/max x values
        advance = max_x - min_x # amount to move forward
        # shift glyph to the left to eliminate empty space, then
        # add the current cursor value
        glyph_path = glyph_path.translate(-min_x + cx, 0)
        page.place(pen.path(glyph_path))
    cx += advance + kern
show(page)

For some of the code below, it'll be helpful to typeset text into a larger `Path` object, instead of placing it on the page directly. (This will make it easier to move the typeset text around on the page.) The following function has largely the same logic as the code above, but returns a composite `Path` with all of the characters in the given string at their correct positions:

In [ ]:
from itertools import chain
def combine_path(alpha, s, kern=0):
    text_paths = []
    cx = 0
    for ch in s:
        if ch == ' ':
            advance = 5
        else:
            glyph_path = alpha[c2i(ch)]
            min_x, max_x = x_extent(glyph_path)
            advance = max_x - min_x
            glyph_path = glyph_path.translate(-min_x + cx, 0)
            text_paths.append(glyph_path)
        cx += advance + kern
    combined = Path(list(chain(*text_paths)))
    return combined

A quick demo to make sure it works. (Rotated so that you can see that rotation affects the entire string, not individual characters.)

In [ ]:
string_path = combine_path(alpha, s)
page = document(300, 50, 'mm').addpage()
pen = shape().nostroke().fill(rgb(40, 40, 40))
page.place(pen.path(string_path.rotate(-np.pi / 12)))
show(page)

## Interpolating between alphabets

Generative models have an interesting property, which is that outputs generated from similar latent variables produce similar outputs. With the SVG-VAE, this property makes it possible to generate two different alphabets and then *interpolate* between them by generating alphabets from points along the line that connects the original two values.

It might be easier to reason about how interpolation works if we start with single values, instead of vectors. For example, the following bit of code uses numpy's `linspace()` function to generate a list of values that starts at `0` and goes to `100`, with seven stops along the way:

In [ ]:
np.linspace(0, 100, 7)

array([  0.        ,  16.66666667,  33.33333333,  50.        ,
        66.66666667,  83.33333333, 100.        ])

The function will also generate values along the line that connects two 2D points:

In [ ]:
np.linspace([0, 10], [30, 80], 13)

array([[ 0.        , 10.        ],
       [ 2.5       , 15.83333333],
       [ 5.        , 21.66666667],
       [ 7.5       , 27.5       ],
       [10.        , 33.33333333],
       [12.5       , 39.16666667],
       [15.        , 45.        ],
       [17.5       , 50.83333333],
       [20.        , 56.66666667],
       [22.5       , 62.5       ],
       [25.        , 68.33333333],
       [27.5       , 74.16666667],
       [30.        , 80.        ]])

You can use the same function to generate points along the line that connects vectors with a larger number of dimensions. In the cell below, I generate two random vectors, and then use `linspace()` to calculate values between them:

In [ ]:
start = np.random.randn(32)
stop = np.random.randn(32)
latents = np.linspace(start, stop, 8)

The code in the cell below generates alphabets from each of these values. It takes a while to run! You'll notice that—while the first and last samples are very different—each line is similar in the style to the one that follows it. Seen together, it seems as though the first sample is "smoothly transitioning" to the final sample.

In [ ]:
page = document(240, 160, 'mm').addpage()
pen = shape().nostroke().fill(rgb(40, 40, 40))
cy = 0
for i, vec in enumerate(latents):
    print(i, end=" ") # print progress, since this takes a while
    alpha = generate_alphabet(vec)
    string_path = combine_path(alpha, "abcdefghijklmnopqrstuvwxyz", kern=2)
    page.place(pen.path(string_path.scale(0.8).translate(0, cy)))
    cy += 19 # just eyeballed this value, adjust to taste
show(page)

0 1 2 3 4 5 6 7 